In [1]:
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import numpy as np
from sklearn.metrics import mean_squared_error

## Question 1

In [2]:
data = pd.DataFrame(
{'x1':[89.09,84.24,98.77,95.44,90.98,97.39,89.27,88.51,97.06,84.45],
 'x2':[78.48,70.56,93.52,86.72,79.2,91.36,80,76.96,92.56,66.4],
 'y':[113.27,109.77,130.08,120.45,115.09,125.37,116.22,112.08,127.85,107.61]})

In [3]:
data

,x1,x2,y
0,89.09,78.48,113.27
1,84.24,70.56,109.77
2,98.77,93.52,130.08
3,95.44,86.72,120.45
4,90.98,79.20,115.09
5,97.39,91.36,125.37
6,89.27,80.00,116.22
7,88.51,76.96,112.08
8,97.06,92.56,127.85
9,84.45,66.40,107.61


### Part F

In [4]:
def calculateB1B2(dataset):
    sumyx1 = 0
    sumx2square = 0
    sumyx2 = 0
    sumx1x2 = 0
    sumx1square = 0
    for index, row in dataset.iterrows():
        sumyx1 = sumyx1 + (row['y'] * row['x1'])
        sumx2square = sumx2square + (row['x2'] * row['x2'])
        sumyx2 = sumyx2 + (row['y'] * row['x2'])
        sumx1x2 = sumx1x2 + (row['x1'] * row['x2'])
        sumx1square = sumx1square + (row['x1'] * row['x1'])
    b1 = ((sumyx1 * sumx2square) - (sumyx2 * sumx1x2))/((sumx1square * sumx2square) - (sumx1x2 * sumx1x2))
    
    b2 = ((sumyx2 * sumx1square) - (sumyx1 * sumx1x2))/ ((sumx1square * sumx2square) - (sumx1x2 * sumx1x2))
    
    print('b1: ' + str(b1))
    print('b2: ' + str(b2))

In [5]:
calculateB1B2(data)

b1: 1.093501198438671
b2: 0.2168336356292473


### Part G

In [6]:
X = data[['x1','x2']]
Y = data['y']

In [7]:
(data['x1'] * data['y']).sum()

108154.6169

In [8]:
regr = linear_model.LinearRegression(fit_intercept=False)
regr.fit(X, Y)

LinearRegression(fit_intercept=False)

In [9]:
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

Intercept: 
 0.0
Coefficients: 
 [1.0935012  0.21683364]


## Question 3

In [10]:
physical_law = pd.read_csv('PhysicalLaw.csv')

In [11]:
physical_law

,p1,p2,d,F
0,4345.7,1510000.0,2.0477,0.073496
1,443000.0,760000.0,13.5380,0.008592
2,503000.0,1060000.0,21.7020,0.001035
3,505000.0,198000.0,1.8970,1.320800
4,2030000.0,259000.0,4.5553,0.713990
...,...,...,...,...
235,424000.0,1450000.0,12.1770,0.025195
236,1230000.0,2390000.0,7.2216,0.922420
237,678000.0,1680000.0,10.1920,0.098119
238,917000.0,1270000.0,5.7909,0.756070


In [12]:
physical_X = physical_law[['p1','p2','d']]
physical_y = physical_law['F']

In [13]:
physical_X_train, physical_X_test, physical_y_train, physical_y_test = train_test_split(physical_X, physical_y, test_size=40, random_state=1)

In [14]:
physical_regr = linear_model.LinearRegression()
physical_regression = physical_regr.fit(physical_X_train,physical_y_train)

In [15]:
physical_model = sm.OLS(physical_y_train,physical_X_train)

In [16]:
result = physical_model.fit()

In [17]:
print(result.summary())

                                 OLS Regression Results                                
Dep. Variable:                      F   R-squared (uncentered):                   0.153
Model:                            OLS   Adj. R-squared (uncentered):              0.140
Method:                 Least Squares   F-statistic:                              11.82
Date:                Wed, 27 Jan 2021   Prob (F-statistic):                    3.77e-07
Time:                        18:03:43   Log-Likelihood:                         -1107.7
No. Observations:                 200   AIC:                                      2221.
Df Residuals:                     197   BIC:                                      2231.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [18]:
physical_y_pred = physical_regr.predict(physical_X_test)

In [19]:
rms = np.sqrt(mean_squared_error(physical_y_test,physical_y_pred))

In [20]:
print(rms)

29.56770293145746


## Question 4

In [21]:
modelselection_data = pd.read_csv('ModelSelection.csv')

In [28]:
modelselection_data['x2'] = modelselection_data['x']
modelselection_data['x3'] = modelselection_data['x']

In [61]:
modelselection_data.head()

,x,y,x2,x3
0,0.997399,0.002905,0.997399,0.997399
1,0.621641,0.346669,0.621641,0.621641
2,0.715032,0.127638,0.715032,0.715032
3,0.463848,0.683570,0.463848,0.463848
4,0.650216,0.288742,0.650216,0.650216


In [62]:
bestMSE = 1
best_model_coef = []
best_model_nvalues = []

In [63]:
for n1 in range(1,11):
    for n2 in range(1,11):
        for n3 in range(1,11):
            #Copy
            adjusted_modelselection_data = modelselection_data.copy()
            
            #Applying exponents
            adjusted_modelselection_data['x'] = adjusted_modelselection_data['x']**n1
            adjusted_modelselection_data['x2'] = adjusted_modelselection_data['x2']**n2
            adjusted_modelselection_data['x3'] = adjusted_modelselection_data['x3']**n2
            
            #Separating into X and y
            adjusted_model_X = adjusted_modelselection_data[['x','x2','x3']]
            adjusted_model_y = adjusted_modelselection_data['y']
            
            #Splitting into training
            model_X_train, model_X_test, model_y_train, model_y_test = train_test_split(adjusted_model_X, adjusted_model_y, test_size=500, random_state=1)

            # Running Regression Model
            modelselection_regr = linear_model.LinearRegression()
            modelselection_regression = modelselection_regr.fit(model_X_train,model_y_train)
            modelselection_ypred = modelselection_regression.predict(model_X_test)
            MSE = mean_squared_error(model_y_test,modelselection_ypred)
            
            #Capturing best MSE, coefficients, and nvalues
            if(MSE < bestMSE):
                bestMSE = MSE
                best_model_coef = modelselection_regression.coef_
                best_model_nvalues = [n1,n2,n3]

In [64]:
print('Best MSE: ' + str(bestMSE))
print('Best Coefficients: ' + str(best_model_coef))
print('Best n values: ' + str(best_model_nvalues))

Best MSE: 0.05227941576034468
Best Coefficients: [ 6.90329000e-01 -7.88317803e+12  7.88317803e+12]
Best n values: [9, 4, 1]
